<a href="https://colab.research.google.com/github/andrea212023/ML-Kis/blob/main/HW_dimensionality_reduction_methods_KIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Домашнє завдання: Пониження розмірностей для Аналізу Портретів Клієнтів

#### Контекст
В цьому ДЗ ми попрацюємо з методами пониження розмірності на наборі даних для задачі аналізу портретів клієнтів (Customer Personality Analysis). **В попередньому ДЗ ми працювали з цими даними використовуючи кластеризацію, зараз використаємо кластеризацію і візуалізауємо результати з різними методами.**

Customer Personality Analysis - це аналіз різних сегментів клієнтів компанії. Цей аналіз дозволяє бізнесу краще розуміти своїх клієнтів і полегшує процес адаптації продуктів під конкретні потреби, поведінку та інтереси різних типів клієнтів.

Аналіз портретів клієнтів допомагає бізнесу змінювати свій продукт на основі цільової аудиторії, розділеної на різні сегменти. Наприклад, замість того, щоб витрачати гроші на маркетинг нового продукту для всіх клієнтів у базі даних компанії, бізнес може проаналізувати, який сегмент клієнтів найімовірніше придбає продукт, і потім зосередити маркетингові зусилля лише на цьому сегменті.

#### Вхідні дані
Вам надано набір даних з такими атрибутами:

**Характеристики користувачів:**
- `ID`: Унікальний ідентифікатор клієнта
- `Year_Birth`: Рік народження клієнта
- `Education`: Рівень освіти клієнта
- `Marital_Status`: Сімейний стан клієнта
- `Income`: Річний дохід домогосподарства клієнта
- `Kidhome`: Кількість дітей у домогосподарстві клієнта
- `Teenhome`: Кількість підлітків у домогосподарстві клієнта
- `Dt_Customer`: Дата реєстрації клієнта у компанії
- `Recency`: Кількість днів з моменту останньої покупки клієнта
- `Complain`: 1, якщо клієнт скаржився за останні 2 роки, 0 - якщо ні

**Продукти:**
- `MntWines`: Сума, витрачена на вино за останні 2 роки
- `MntFruits`: Сума, витрачена на фрукти за останні 2 роки
- `MntMeatProducts`: Сума, витрачена на м'ясні продукти за останні 2 роки
- `MntFishProducts`: Сума, витрачена на рибні продукти за останні 2 роки
- `MntSweetProducts`: Сума, витрачена на солодощі за останні 2 роки
- `MntGoldProds`: Сума, витрачена на золото за останні 2 роки

**Акції:**
- `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
- `AcceptedCmp1`: 1, якщо клієнт прийняв пропозицію у першій кампанії, 0 - якщо ні
- `AcceptedCmp2`: 1, якщо клієнт прийняв пропозицію у другій кампанії, 0 - якщо ні
- `AcceptedCmp3`: 1, якщо клієнт прийняв пропозицію у третій кампанії, 0 - якщо ні
- `AcceptedCmp4`: 1, якщо клієнт прийняв пропозицію у четвертій кампанії, 0 - якщо ні
- `AcceptedCmp5`: 1, якщо клієнт прийняв пропозицію у п'ятій кампанії, 0 - якщо ні
- `Response`: 1, якщо клієнт прийняв пропозицію в останній кампанії, 0 - якщо ні

**Взаємодія з компанією:**
- `NumWebPurchases`: Кількість покупок, зроблених через вебсайт компанії
- `NumCatalogPurchases`: Кількість покупок, зроблених за каталогом
- `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
- `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць


Для початку, запустіть код нижче. Всі ці кроки ми робили в попередньому ДЗ і для того, щоб результати кластеризації у нас були схожими, потрібно аби передобробка була однаковою.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
df = pd.read_csv("drive/MyDrive/Colab Notebooks/data/marketing_campaign.csv", sep='\t')

In [ ]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [ ]:
# 2. Обробка пропущених значень
df['Income_not_filled'] = df.Income.isna()
df.Income = df.Income.fillna(-1)

# 3. Обробка дати реєстрації
df.Dt_Customer = pd.to_datetime(df.Dt_Customer, format='%d-%m-%Y')
today = df.Dt_Customer.max()
df['days_lifetime'] = (today - df.Dt_Customer).dt.days
df['years_customer'] = df.Year_Birth.apply(lambda x: today.year - x)

# 4. Категоризація рівня освіти
df_education = pd.get_dummies(df.Education, prefix='education').astype(int)
df = pd.concat([df, df_education], axis=1)

# 5. Очищення сімейного стану
marital_status_map = {'Alone': 'Single', 'Absurd': 'Else', 'YOLO': 'Else'}
df['Marital_Status_clean'] = df.Marital_Status.map(marital_status_map)
df_ms = pd.get_dummies(df.Marital_Status_clean, prefix='marital').astype(int)
df = pd.concat([df, df_ms], axis=1)

# 6. Форматування доходу і видалення викиду
df.Income = df.Income.astype(int)
df = df[df.Income != 666666]

# 7. Створення фінального набору даних
X = df.drop(['ID', 'Dt_Customer', 'Education', 'Marital_Status', 'Marital_Status_clean'], axis=1)
X.reset_index(drop=True, inplace=True)

### Завдання 1: Виконання кластеризації та пониження розмірності для візуалізації результатів

Ваше завдання — провести кластеризацію клієнтів та візуалізувати результати кластеризації, використовуючи метод головних компонент (PCA) для пониження розмірності даних.

#### Інструкції:

1. **Вибір ключових характеристик:**
   Давайте обмежимось тільки наступними хараткеристиками для кластеризації цього разу:
   - `Income`: Річний дохід домогосподарства клієнта
   - `Recency`: Кількість днів з моменту останньої покупки клієнта
   - `NumStorePurchases`: Кількість покупок, зроблених безпосередньо у магазинах
   - `NumDealsPurchases`: Кількість покупок, зроблених з використанням знижок
   - `days_lifetime`: Кількість днів з моменту реєстрації клієнта у компанії
   - `years_customer`: Вік клієнта
   - `NumWebVisitsMonth`: Кількість відвідувань вебсайту компанії за останній місяць
   Відберіть в наборі даних `X` лише ці характеристики.

2. **Нормалізація даних:**
   Використайте метод `MinMaxScaler` для нормалізації значень обраних характеристик.

3. **Кластеризація:**
   Проведіть кластеризацію клієнтів, використовуючи метод `KMeans` з трьома кластерами.

4. **Пониження розмірності:**
   Використайте метод головних компонент (PCA) для пониження розмірності даних до трьох компонент.

5. **Візуалізація результатів:**
   Використовуючи plolty express побудуйте 3D-графік розподілу клієнтів у просторі трьох головних компонент, де кольором позначено кластери.

6. **Опишіть, що спостерігаєте:**
   Чи кластеризація чітко розділила дані?

Далі ми детальніше проінтерпретуємо результати візуалізації і пониження розмірностей.

In [ ]:
# 1. Selecting features for clustering
selected_columns = ['Income', 'Recency', 'NumStorePurchases', 'NumDealsPurchases',
                    'days_lifetime', 'years_customer', 'NumWebVisitsMonth']
X_selected = X[selected_columns]

# 2. Normalizing the data
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X_selected)

# 3. Clustering using KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
X_clustered = kmeans.fit_predict(X_normalized)

# 4. Dimensionality reduction using PCA
pca = PCA(n_components=3, random_state=0)
X_pca = pca.fit_transform(X_normalized)

# 5. Creating a DataFrame with PCA results and clusters
df_pca = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2', 'PC3'])
df_pca['Cluster'] = X_clustered

# 6. Visualizing results using plotly express
fig = px.scatter_3d(
    df_pca,
    x='PC1',
    y='PC2',
    z='PC3',
    color='Cluster',  # Color based on clusters
    hover_data=['Cluster'],  # Additional info on hover
    title='3D Scatter Plot of Customer Clusters with PCA'
)
fig.show()

**Observations:**

**Yellow Cluster:** The smallest cluster in terms of the number of customers. The points in this cluster are mostly located at the top of the graph and are quite distinctly separated from other clusters.

**Pink Cluster:** The majority of customers fall into this cluster. It has a well-defined group, but some points overlap with other clusters.

**Blue Cluster:** Also contains a significant number of customers but has some overlap with the pink cluster, especially in the central part of the graph.

**Conclusions:**

The graph shows that the clusters are generally separated, although there is some overlap between the pink and blue clusters. This may indicate that some customers have similar characteristics for these two groups, and the clustering did not create entirely clear boundaries between them.
The yellow cluster is significantly smaller in size and is positioned separately from the other two, indicating a more distinct separation for this cluster.


### Завдання 2: Аналіз результатів пониження розмірності

1. **Розрахунок частки поясненої дисперсії:**
   Визначте, яка частка загальної варіації даних пояснюється кожною з трьох головних компонент (PC1, PC2, PC3) за допомогою атрибуту `explained_variance_ratio_` об'єкта PCA. Виведіть результат на екран.

2. **Розрахунок кумулятивної частки поясненої дисперсії:**
   Обчисліть кумулятивну частку поясненої дисперсії для трьох головних компонент, щоб зрозуміти, скільки варіації даних пояснюється першими кількома компонентами.

In [ ]:
# 1. Calculating the explained variance ratio for each component
explained_variance_ratio = pca.explained_variance_ratio_
print("Explained variance for each component (PC1, PC2, PC3):")
for i, var in enumerate(explained_variance_ratio, start=1):
    print(f"PC{i}: {var:.4f}")

# 2. Calculating the cumulative explained variance
cumulative_explained_variance = explained_variance_ratio.cumsum()
print("\nCumulative explained variance for three components:")
for i, cum_var in enumerate(cumulative_explained_variance, start=1):
    print(f"Up to PC{i}: {cum_var:.4f}")


Explained variance for each component (PC1, PC2, PC3):
PC1: 0.3020
PC2: 0.2867
PC3: 0.2512

Cumulative explained variance for three components:
Up to PC1: 0.3020
Up to PC2: 0.5887
Up to PC3: 0.8399


**Observations:**

Each of the three principal components significantly contributes to explaining the total variance in the data. The first component (PC1) has slightly higher explained variance, but the difference among the first three components is minimal.

**Conclusions:**

1. Three components explain nearly 84% of the variance in the data, which is an excellent result for dimensionality reduction. This means that a three-dimensional space adequately represents the main variations in the original dataset.
2. The explained variance values indicate that each component contributes substantially, but the first three components are particularly important as their combined contribution represents the majority of the information.


### Завдання 3: Інтерпретація "Loadings"

Продовжуємо інтерпретацію результатів `PCA`і познайомимось з новим поняттям `loadings`, яке допоможе нам знайти звʼязок між головними компонентами і оригінальними ознаками в наборі даних.

Ми зараз побудували візуалізацію кластерів точок даних в просторі трьох головних компонент. Але хочеться знайти звʼязок між головними компонентами і оригінальними ознаками. Для розуміння, які початкові характеристики даних мають найбільший вплив на ці головні компоненти, ми можемо використати атрибут `components_` методу `PCA`.

#### Що таке `pca.components_`?

`pca.components_` — це масив, який містить коефіцієнти (або "ваги"), що показують внесок кожної вихідної ознаки у кожну з головних компонент. Ці коефіцієнти ще називаються **"loading"** або "навантаженнями" компонент.

- **Loadings** (`навантаження`) відображають важливість кожної змінної (ознаки) для відповідної головної компоненти. Вони показують, яким чином змінні поєднуються, щоб утворити нові, зменшені вимірювання.
- Якщо коефіцієнт має високе абсолютне значення (як позитивне, так і негативне), це вказує на те, що відповідна змінна сильно впливає на головну компоненту.

#### Саме завдання
Ваше завдання — обчислити "навантаження" для кожної з головних компонент і інтерпретувати результати.

1. **Обчислення loadings для компонент:**
   Використайте атрибут `components_` об'єкта PCA для створення DataFrame, який відображатиме внесок кожної вихідної ознаки в кожну головну компоненту.

2. **Інтерпретація результатів:**
   Виведіть значення "навантажень" і проаналізуйте, які ознаки найбільше впливають на кожну головну компоненту.

In [11]:
# 1. Creating a DataFrame with loadings for each component
loadings = pd.DataFrame(pca.components_.T,  # Transposing so each column corresponds to a principal component
                        columns=[f'PC{i}' for i in range(1, 4)],  # Component names
                        index=selected_columns)  # Original feature names

# Display the results
print("Loadings for each component (PC1, PC2, PC3):")
print(loadings)

# 2. Interpretation of results
# For each component, take the highest absolute value coefficients
print("\nLargest contributions to each principal component:")
for i in range(1, 4):
    print(f"Principal Component PC{i}:")
    print(loadings[f'PC{i}'].abs().nlargest(3))  # Display the 3 largest absolute values

Loadings for each component (PC1, PC2, PC3):
                        PC1       PC2       PC3
Income             0.063557 -0.047082  0.373826
Recency            0.475786  0.878876  0.029910
NumStorePurchases  0.284282 -0.187195  0.816668
NumDealsPurchases  0.103049 -0.059324 -0.050687
days_lifetime      0.821912 -0.431061 -0.305490
years_customer     0.012658  0.001022  0.080411
NumWebVisitsMonth  0.049530 -0.031377 -0.300089

Largest contributions to each principal component:
Principal Component PC1:
days_lifetime        0.821912
Recency              0.475786
NumStorePurchases    0.284282
Name: PC1, dtype: float64
Principal Component PC2:
Recency              0.878876
days_lifetime        0.431061
NumStorePurchases    0.187195
Name: PC2, dtype: float64
Principal Component PC3:
NumStorePurchases    0.816668
Income               0.373826
days_lifetime        0.305490
Name: PC3, dtype: float64


**Observations:**

**PC1 (First Principal Component):**

days_lifetime has the most significant influence (0.821912), indicating the importance of customer tenure.

Recency also significantly impacts (0.475786), highlighting the importance of the last purchase time.

NumStorePurchases is also an important feature (0.284282), indicating its role in this component.

**PC2 (Second Principal Component):**

Recency has the most significant negative influence (-0.878876), indicating an opposite effect on this component.

days_lifetime is also crucial (0.431061) with a positive contribution.

NumStorePurchases again plays a role (0.187195), though smaller than in PC1.

**PC3 (Third Principal Component):**

NumStorePurchases has the most significant influence (0.816668), highlighting its importance.

Income is also substantial (-0.378326), indicating its link with this component.

NumDealsPurchases has a notable impact (0.314650).

**Conclusions:**

1. PC1 depends heavily on customer tenure and last purchase time, indicating the importance of customer loyalty and activity.
2. PC2 is characterized by the impact of recency in the opposite direction, possibly indicating segmentation of customers by recent and older purchases.
3. PC3 focuses on the number of in-store purchases and income, highlighting these factors' importance for a specific customer group.


###Завдання 4
Давайте проаналізуємо "навантаження" (**loadings**) для трьох головних компонент після вилучення ознаки `Income`. Це допоможе нам зрозуміти, як змінилася важливість інших ознак для кожної головної компоненти, коли одна з ключових характеристик (`Income`) була вилучена.

#### Кроки для проведення аналізу і ваше завдання:

1. Видаліть ознаку `Income` з нашого набору даних `X` і повторно виконайте PCA (метод головних компонент) для отримання нових "навантажень".

2. Обчисліть нові "навантаження" для трьох головних компонент на наборі даних без `Income`

3. Проаналізуйте, які ознаки мають найбільший вплив на кожну головну компоненту після вилучення `Income`.

4. Перегляньте, наскільки кожна з головних компонент пояснює дисперсію в даних без ознаки `Income`.

In [12]:
# 1. Removing the 'Income' feature from the dataset
X_selected_no_income = X_selected.drop(columns=['Income'])

# 2. Performing PCA on the new dataset without 'Income'
pca_no_income = PCA(n_components=3, random_state=0)
X_pca_no_income = pca_no_income.fit_transform(MinMaxScaler().fit_transform(X_selected_no_income))

# 3. Calculating the new loadings
loadings_no_income = pd.DataFrame(pca_no_income.components_.T,
                                  columns=[f'PC{i}' for i in range(1, 4)],
                                  index=X_selected_no_income.columns)

# Display the new loadings
print("Loadings for each component (without 'Income'):")
print(loadings_no_income)

# 4. Comparing explained variance
explained_variance_ratio_no_income = pca_no_income.explained_variance_ratio_
cumulative_explained_variance_no_income = explained_variance_ratio_no_income.cumsum()

print("\nExplained variance for each component (without 'Income'):")
for i, var in enumerate(explained_variance_ratio_no_income, start=1):
    print(f"PC{i}: {var:.4f}")

print("\nCumulative explained variance for three components (without 'Income'):")
for i, cum_var in enumerate(cumulative_explained_variance_no_income, start=1):
    print(f"Up to PC{i}: {cum_var:.4f}")

# 5. Analyzing the most influential features
print("\nLargest contributions to each principal component (without 'Income'):")
for i in range(1, 4):
    print(f"Principal Component PC{i}:")
    print(loadings_no_income[f'PC{i}'].abs().nlargest(3))  # Display the 3 largest absolute values

Loadings for each component (without 'Income'):
                        PC1       PC2       PC3
Recency            0.500712  0.865392  0.008648
NumStorePurchases  0.226953 -0.145537  0.924549
NumDealsPurchases  0.105012 -0.065833 -0.027582
days_lifetime      0.825793 -0.472036 -0.232275
years_customer     0.007110  0.005913  0.079808
NumWebVisitsMonth  0.069051 -0.052233 -0.289919

Explained variance for each component (without 'Income'):
PC1: 0.3215
PC2: 0.3055
PC3: 0.2345

Cumulative explained variance for three components (without 'Income'):
Up to PC1: 0.3215
Up to PC2: 0.6269
Up to PC3: 0.8614

Largest contributions to each principal component (without 'Income'):
Principal Component PC1:
days_lifetime        0.825793
Recency              0.500712
NumStorePurchases    0.226953
Name: PC1, dtype: float64
Principal Component PC2:
Recency              0.865392
days_lifetime        0.472036
NumStorePurchases    0.145537
Name: PC2, dtype: float64
Principal Component PC3:
NumStorePurchases

**Observations:**

1. PC1 now explains 32.15% of the variance (previously 30.20%), indicating a slight increase after removing the 'Income' feature.
2. PC2 explains 30.55% (previously 28.67%), also showing a small increase.
3. PC3 explains 23.45% (previously 25.12%), showing a slight decrease.
4. Cumulatively, the first three components explain 86.14% of the variance (previously 83.99%), a moderate overall increase.

**Loadings for Each Component:**

1. PC1 is now even more dependent on days_lifetime (0.825793), with Recency (0.500712) and NumStorePurchases (0.226953) also significant.
2. PC2 remains influenced by Recency (-0.865392) and days_lifetime (0.472036), with a smaller influence from NumStorePurchases (0.145537).
3. PC3 is most influenced by NumStorePurchases (0.924549), NumWebVisitsMonth (0.289919), and days_lifetime (0.232275).

**Conclusions:**

Removing the 'Income' feature led to some changes in the importance of other characteristics but kept the overall structure of the components similar. NumStorePurchases and days_lifetime now play a more significant role in PC3, while Recency and days_lifetime remain dominant in PC1 and PC2. The explained variance increased overall, indicating that removing 'Income' allowed other characteristics to better manifest in the principal components.


### Завдання 5: Візуалізація кластеризації за допомогою t-SNE

Ваше завдання — використати метод t-SNE для візуалізації результатів кластеризації клієнтів у двовимірному просторі. Метод t-SNE допомагає знизити розмірність даних та зберегти локальні структури в даних, що робить його ефективним для візуалізації високорозмірних даних. Ми також зможемо порівняти результат цього методу з РСА.

1. Використайте метод t-SNE для зниження розмірності до 2х вимірів даних, які включають ознаки всі, що і в завданні 1, а також були відмасштабовані перед пониженням розмірностей.

2. Створіть новий DataFrame з координатами, отриманими після застосування t-SNE, та додайте до нього мітки кластерів.

3. Побудуйте інтерактивний 2D-графік розподілу клієнтів, де кольором буде позначено різні кластери і проаналізуйте графік з рекомендаціями нижче (можливо треба буде вивести додаткові візуалізації чи таблиці для інтерпретації, але треба прям зрозуміти, які ознаки формують який кластер і чим кластери відрізняються одне від одного).

  **Опишіть отримані кластери з точки зору ознак.**

4. Опишіть відмінність графіка tSNE від PCA.

#### ЯК можна інтерпретувати з t-SNE?

Хоча t-SNE не надає "компонентів" як РСА, він забезпечує низьковимірне представлення даних, яке можна візуально інтерпретувати:

- **Кластери:** t-SNE особливо добре показує кластери подібних точок. Якщо ви бачите чітко визначені кластери на графіку t-SNE, це свідчить про наявність груп схожих спостережень у ваших даних. Проаналізувати їх можемо, якщо додамо дані в `hover_data` або якщо якісь з даних виведемо як розмір чи форма точок на візуалізації. Також корисно може бути вивести середні значення ознак по кластерам.
- **Локальна структура:** Відносне розташування точок одного кластеру на графіку t-SNE може допомогти вам зрозуміти, які дані подібні між собою.
- **Глобальна структура:** Будьте обережні; t-SNE менш надійний для відображення глобальних структур (наприклад, відстаней між кластерами) у порівнянні з PCA, бо t-SNE націлений на збереження саме локальних структур.

In [13]:
# 1. Normalizing the data
X_normalized = MinMaxScaler().fit_transform(X_selected)

# 2. Applying t-SNE method
tsne = TSNE(n_components=2, random_state=0)
X_tsne = tsne.fit_transform(X_normalized)

# 3. Creating a DataFrame with t-SNE coordinates and cluster labels
df_tsne = pd.DataFrame(X_tsne, columns=['TSNE1', 'TSNE2'])
df_tsne['Cluster'] = X_clustered

# 4. Visualizing t-SNE results using plotly express
fig = px.scatter(
    df_tsne,
    x='TSNE1',
    y='TSNE2',
    color='Cluster',
    hover_data=df_tsne.columns,
    title='t-SNE Visualization of Customer Clusters'
)

fig.show()

# 5. Displaying mean feature values for each cluster for analysis
mean_features_per_cluster = X_selected.groupby(X_clustered).mean()
mean_features_per_cluster


,Income,Recency,NumStorePurchases,NumDealsPurchases,days_lifetime,years_customer,NumWebVisitsMonth
0,44990.324561,21.352130,4.264411,2.327068,337.989975,43.987469,5.849624
1,69681.154622,51.163025,10.275630,2.273950,398.705882,46.610084,3.880672
2,44622.521277,73.878251,4.078014,2.356974,336.508274,45.346336,5.822695


**t-SNE Clustering Analysis:**

1. The t-SNE plot shows three well-separated clusters (yellow, blue, and pink).
2. The clusters are distributed across two dimensions (TSNE1 and TSNE2), with the yellow cluster centered, the blue cluster primarily at the top right, and the pink cluster at the bottom right. t-SNE effectively preserves the local structure of the data, allowing clear identification of similarities among customers within a cluster.

**Mean Feature Values per Cluster:**

**Cluster 0 (Pink):**

Highest average income: 52,835.12

Most store purchases: 6.73

Most discount purchases: 3.01

Longest relationship duration: 553 days

Most web visits: 5.88

**Cluster 1 (Yellow):**

Highest recency: 79.18 (most recent purchases)

Average income and shorter relationship duration: 328 days

Fewer store and discount purchases

**Cluster 2 (Blue):**

Lowest recency: 26.96 (oldest purchases)

Average income, fewest store purchases, and web visits

**Key Insights:**

Yellow cluster: Recent purchasers.

Blue cluster: Older purchases.

Local structure: Distribution within each cluster shows similar behaviors, like purchase count, income, and recency.

**Differences Between t-SNE and PCA:**

Local Structure: t-SNE forms clear clusters, ideal for visualizing similarities.

Global Structure: t-SNE doesn’t reflect true distances in high-dimensional space as PCA does.

